# [WIP] Pipe mode for Chainer conatiner with PipeModeDataset

PipeモードでS3においたTFRecordデータをトレーニングインスタンスに送信して学習します。ChainerはTFRecordをパースする関数を持っていないので、PipeModeDatasetで読み込んでChainerに入力して学習します。

PipeModeDataset, https://github.com/aws/sagemaker-tensorflow-extensions 

## The dataset
The [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) is one of the most popular machine learning datasets. It consists of 60,000 32x32 images belonging to 10 different classes (6,000 images per class). Here are the classes in the dataset, as well as 10 random images from each:

![cifar10](https://maet3608.github.io/nuts-ml/_images/cifar10.png)

In this tutorial, we will train a deep CNN to recognize these images.

We'll compare trainig with file mode, pipe mode datasets and distributed training with Horovod

## Set up the environment

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

## Download the CIFAR-10 dataset
Downloading the test and training data takes around 5 minutes.

In [ ]:
!python generate_cifar10_tfrecords.py --data-dir ./data

## Run on SageMaker cloud

### Uploading the data to s3

In [ ]:
dataset_location = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-cifar10-tf')
display(dataset_location)

### Train image classification based on the cifar10 dataset

In [ ]:
hyperparameters = {'epochs': 2, 'batch-size' : 256}

## Run on SageMaker with Pipe Mode input
SageMaker Pipe Mode is a mechanism for providing S3 data to a training job via Linux fifos. Training programs can read from the fifo and get high-throughput data transfer from S3, without managing the S3 access in the program itself.
Pipe Mode is covered in more detail in the SageMaker [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html#your-algorithms-training-algo-running-container-inputdataconfig)

in out script, we enabled Pipe Mode using the following code:
```python
from sagemaker_tensorflow import PipeModeDataset
dataset = PipeModeDataset(channel=channel_name, record_format='TFRecord')
```

In [ ]:
from sagemaker.chainer import Chainer

source_dir = os.path.join(os.getcwd(), 'source_dir')
estimator_pipe = Chainer(base_job_name='pipe-cifar10',
                       entry_point='cifar10_chainer_main.py',
                       source_dir=source_dir,
                       role=role,
                       framework_version='5.0.0',
                       hyperparameters=hyperparameters,
                       train_instance_count=1, train_instance_type='ml.m4.xlarge',
                       input_mode='Pipe')

In this example, we set ```wait=False``` if you want to see the output logs, change this to ```wait=True```

In [ ]:
remote_inputs = {'train' : dataset_location+'/train', 'validation' : dataset_location+'/validation', 'eval' : dataset_location+'/eval'}
estimator_pipe.fit(remote_inputs)